1.   학습률 : 일반적으로 훈련 알고리즘이 발산하는 학습률의 절반 or 학습률을 올릴때 손실이 발생하는 지점의 1/10 정도가 적절한 학습률이다. 학습률은 다른
하이퍼 파라미터에 대해 많은 영향을 받으므로, 하이퍼 파라미터를 수정했다면
학습률도 수정해야한다.

2.   배치 크기 : GPU RAM에 맞는 가장 큰 배치 크기를 사용하라고 조언하는 사람이
많지만 한 32개 정도가 처음에 시도해보기 좋은 배치 크기이다. 학습률 예열 같은
기법을 사용하면 배치크기를 8192개 까지 늘리는 등의 기교를 부릴 수 있다.
compile메서드의 steps_per_excution을 1 이상으로 지정하면 한 번에 여러 배치를
처리해 GPU를 최대한으로 활용하여 훈련 속도를 빠르게 할 수 있다.

3. 반복 횟수 : 반복 횟수를 튜닝하기보다는 조기 종료 콜백을 사용한다.

4. 옵티마이저 : 각 데이터셋에 적절한 옵티마이저를 선택해야한다.

5. 가중치 감쇠(weight decay) = 1e-4가 일단은 시작하기 좋은 값처럼 보임

하이퍼 파라미터 튜닝 모범 사례 arxiv 1803.09820 (https://homl.info/1cycle)


# 딥러닝 모델 Pipeline

In [ ]:
import tensorflow as tf

fashion_minst = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_minst
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [ ]:
X_train, X_valid, X_test = X_train / 255. , X_valid / 255., X_test / 255. # 전처리

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
# 가장 기본적인 모델 쌓기
from tensorflow import keras
from keras import layers

tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(28, 28)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

#아래와 같다. 시퀀셜 API
tf.keras.backend.clear_session()
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

#또는 이렇게도 쓸 수 있다. 함수형 API 라고 함.
tf.keras.backend.clear_session()
tf.random.set_seed(42)

inputs = keras.Input(shape = (28,28))
x = layers.Flatten()(inputs)
x = layers.Dense(300, activation = 'relu')(x)
x = layers.Dense(100, activation = 'relu')(x)
outputs = layers.Dense(10, activation = 'softmax')(x)

model = keras.Model(inputs, outputs)

In [ ]:
"""
# 모델을 쌓고나면 컴파일 해야한다
loss = binary_crossentropy : 이진 분류
       sparse_categorical_crossentropy : 다중 분류(원-핫 벡터가 아닐때)
       categorical_crossentropy : 다중 분류 (원-핫 벡터 일때)

optimizer = rmsprop (Root Mean Square Propagation) : 기본값, 적응적인 학습률
            sgd (Stochastic Gradient Descent) : 확률적 경사 하강법
            adma (Adaptive Moment Estimation) : SGD + Rmsprop
            adagrad (Adaptive Gradient Algorithm) : 각 파라미터에 대해 개별적 학습률
            Nadam (Nesterov-accelerated Adaptive Moment Estimation)

metrics = Accuracy : 올바르게 예측한 샘플의 비율
          Precision : 양성 클래스로 예측한 샘플 중 실제 양성의 비율
          Recall : 양성 클래스 샘플 중 양성으로 예측한 샘플의 비율
          F1 Score : 정밀도와 재현율의 조화 평균
          AUC : ROC 곡선 아래의 면적 (1이 최대)

metrcis 추가 지표 : binary_crossentropy,
                    categorical_crossentropy,
                    mean_squared_error
                    True Positives, False Positives, True Negatives, False Negatives
"""


"""
# 추가 코드 - 이 셀은 아래와 같다.
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.SGD(),
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
"""
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'sgd',
              metrics = ['accuracy'])


In [ ]:
"""
# 모델 훈련
class_weight = 각 feature 마다 weight를 할당할 수 있다. (기본 1, 0은 제외를 의미)
               딕셔너리 형태로 제공한다.
sample_weight = 각 Sample 마다 weight를 할당할 수 있다.
batch_size = 기본은 32로 전체 샘플을 32개로 쪼개서 훈련하는 것을 의미함(미니배치)

validation_data = 검증세트를 전달할 수도 있고, validation_split = 0.1처럼 지정해
                  X_train, y_train의 10%를 떼서 검증 세트로 쓸 수도 있다.
                  따로 나누는 것 보다 이게 더 좋은 것 같음.

callbacks = 콜백 리스트를 따로 만들어서 전달하는게 효율적

모델을 훈련한 후 history.history에는 loss ,val_loss(validation을 지정한 경우),
설정한 메트릭에 따라 'accuracy', 'precision' 등이 들어있다.
"""
history = model.fit(X_train, y_train, epochs = 30,
                    validation_data = (X_valid, y_valid))

In [ ]:
#모델을 훈련한 뒤에는 테스트 점수를 계산하면 된다.
#[0]에는 compile에서 지정한 loss가, [1]에는 metrics가 들어있다
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3234 - accuracy: 0.8859


[0.3233807384967804, 0.8859000205993652]

In [ ]:
"""
콜백(Callbacks)
ModelCheckPoint와 Early Stopping을 주로 사용하며
아래와 같이 구현한다.

from keras.callbacks import ModelCheckpoint, EarlyStopping

callbacks = [ModelCheckpoint("my_checkpoints", save_weight_only = True,
                             save_best_only = True),
             EarlyStopping(patience = 10, restore_best_weights = True)
]

history = model.fit(X_train, y_train, epochs = 30,
                    validation_data = (X_valid, y_valid),
                    callbacks = callbacks)


# 사용자 정의 콜백
on_train_begin() / on_train_end() / batch / epoch 등등
다양한 환경에서 데이터를 저장하고 관찰할 수 있다.

class PrintValTrainRatioCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        ratio = logs["val_loss"] / logs["loss"]
        print(f"Epoch={epoch}, val/train={ratio:.2f}")
"""

In [ ]:
"""
chat cpt가 순환 학습률을 구현한 사용자 정의 콜백

from keras.callbacks import Callback
import keras.backend as K

class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular'):
        super().__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.mode == 'triangular':
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x))
        # 기타 모드도 추가 가능

    def on_train_begin(self, logs=None):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

# 모델 생성 및 컴파일
# model = ...

# CyclicLR 콜백 사용 # lr을 0.06 ~ 0.6으로 해야할지도
clr = CyclicLR(base_lr=0.001, max_lr=0.6, step_size=2000., mode='triangular')
model.fit(X_train, Y_train, callbacks=[clr])


"""

In [ ]:
"""
chat cpt가 순환 모멘텀을 구현한 사용자 정의(옵티마이저가 모멘텀 지원해야함)
from keras.callbacks import Callback
import keras.backend as K

class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular'):
        super().__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.mode == 'triangular':
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x))
        # 기타 모드도 추가 가능

    def on_train_begin(self, logs=None):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

# 모델 생성 및 컴파일
# model = ...

# CyclicLR 콜백 사용
clr = CyclicLR(base_lr=0.9, max_lr=0.95, step_size=2000., mode='triangular')
model.fit(X_train, Y_train, callbacks=[clr])

"""

In [ ]:
"""
모델의 저장과 복원은 다음과 같이 한다.
import shutil

#이미 존재하는 경우 삭제
shutil.rmtree("my_keras_model", ignore_errors=True)

model.save("my_keras_model", save_format = "tf") #혹은 "h5"
model = tf.keras.models.load_model("my_keras_model")

model.save_weight(), load_weight()를 활용해 가중치만 저장하고 로드하는 것도 가능

모델을 저장하면 my_keras_model이라는 폴더가 생성되고

keras_metadata.pb에는 케라스에 필요한 추가 정보
variables 폴더에는 모든 파라미터값(가중치,편향,하이퍼 파라미터 등)
assets 폴더에는 데이터 샘플 특성 이름같은 추가 파일이 들어갈 수 있음(기본적으로는 비어있음)
"""

# 시퀀셜 API

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [ ]:
import tensorflow as tf

tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1)
])
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
#norm_layer는 sklearn의 StandardScaler와 같은데, 이처럼 훈련 전에 adapt해야한다.
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test, rmse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

# 함수형 API (다중 입력, 다중 출력)

In [ ]:
"""
함수형 API의 예시

input = layers.Input(shape = X_trian.shape[1:])
normalized = layers.Normalization()(input)
hidden1 = layers.Dense(30, activation = 'relu')(normalized)
hideen2 = layers.Dense(30, activation = 'relu')(hidden1)
concat = layers.Concatenate()([normalized, hidden2])
output = layers.Dense(1)(concat)

model = keras.Model(inputs = input, outputs = output)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
normalization_layer.adapt(X_train) #잊지 말 것
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
y_pred = model.predict(X_new)
"""

In [ ]:
# 다중 입력(특성 건너뛰기)

# 모델 만들기
import tensorflow as tf
from tensorflow import keras
from keras import layers

input_wide = layers.Input(shape=[5])  # 특성 0 ~ 4
norm_wide = layers.Normalization()(input_wide)

input_deep = layers.Input(shape=[6])  # 특성 2 ~ 7
norm_deep = layers.Normalization()(input_deep)
hidden1 = layers.Dense(30, activation="relu")(norm_deep)
hidden2 = layers.Dense(30, activation="relu")(hidden1)

concat = layers.concatenate([norm_wide, hidden2])
output = layers.Dense(1)(concat)

model = keras.Model(inputs=[input_wide, input_deep], outputs=[output])

# 컴파일, 정규화 adapt
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

layers.Normalization().adapt(X_train_wide)
layers.Normalization().adapt(X_train_deep)

# 모델 훈련 및 점수 확인 / 예측
history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20,
                    validation_data=((X_valid_wide, X_valid_deep), y_valid))
mse_loss, rmse = model.evaluate((X_test_wide, X_test_deep), y_test)
y_pred = model.predict((X_new_wide, X_new_deep))

In [ ]:
"""
# 다중 입력과 다중 출력
1. 그림에 있는 물체를 분류하고 위치를 알아야 할 때
2. 동일한 데이터에서 독립적인 여러 작업을 수행할 때,
   예를 들어 같은 얼굴 이미지에서 한 출력은 표정을 출력하고,
   다른 출력은 안경 착용 유무를 출력하는 등.
"""
tf.keras.backend.clear_session()
tf.random.set_seed(42)

input_wide = layers.Input(shape=[5])  # 특성 0 ~ 4
norm_wide = layers.Normalization()(input_wide)

input_deep = layers.Input(shape=[6])  # 특성 2 ~ 7
norm_deep = layers.Normalization()(input_deep)
hidden1 = layers.Dense(30, activation="relu")(norm_deep)
hidden2 = layers.Dense(30, activation="relu")(hidden1)

concat = layers.concatenate([norm_wide, hidden2])
output = layers.Dense(1)(concat)
aux_output = layers.Dense(1)(hidden2)

model = keras.Model(inputs=[input_wide, input_deep],
                    outputs=[output, aux_output])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

#loss를 각각 전달하고, 주 출력에 더 관심이 많다면(보조 출력은 보통 규제로
#사용되므로) 아래와 같이 가중치를 지정해주면 된다.
model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1), optimizer=optimizer,
              metrics=["RootMeanSquaredError"])

layers.Normalization().adapt(X_train_wide)
layers.Normalization().adapt(X_train_deep)

#출력 2개가 y_train으로 같은데 이때 혼동을 방지하기 위해
#모델에 이름을 달고 {"output" : y_train, "aux_output" : y_train}처럼 쓸 수 있다.
history = model.fit(
    (X_train_wide, X_train_deep), # X
    (y_train, y_train), # Y
    epochs=20,
    validation_data= ((X_valid_wide, X_valid_deep),
                      (y_valid, y_valid))
)

#이때 손실은 두 출력의 합과 각각 출력의 손실,metrics가 나온다
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results

# 서브클래싱 API (연구용)

In [ ]:
"""
서브클래싱 API는 저수준 텐서플로르 구현해 갖가지 for문 if문등을
구현할 수 있다는 장점이 있지만 동시에 문제를 추적하거나 모델을 해석하는것을
어렵게 하므로 왠만해서는 시퀀셜 API나 함수형 API를 사용하는 편이 낫다.
"""
class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)  # 모델 이름을 지정하는 데 필요합니다
        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)

    def call(self, inputs):
        input_wide, input_deep = inputs
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden1(norm_deep)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([norm_wide, hidden2])
        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return output, aux_output

tf.random.set_seed(42)  # 추가 코드 - 재현성을 위한 것
model = WideAndDeepModel(30, activation="relu", name="my_cool_model")

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=optimizer,
              metrics=["RootMeanSquaredError"])
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=10,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)))
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

# 텐서보드 (시각화)

In [ ]:
#코랩에서 텐서보드 설치
%pip install -q -U tensorboard-plugin-profile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.1 MB/s eta 0:00:00


In [ ]:
#기본 설정
from pathlib import Path
from time import strftime
import tensorflow as tf

#파일 이름 설정
def get_run_logdir(root_logdir = "my_logs"):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

run_logdir = get_run_logdir()

#텐서보드 콜백. 에포크동안 100~200배치의 신경망을 프로파일링 한다.
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir,
                                                profile_batch = (100, 200))
# 콜백에 담아 훈련한다.
# history = model.fit([...], callbacks = tensorboard_cb)

#텐서보드 열기
%load_ext tensorboard
%tensorboard --logdir=./my_logs #콜백 파일 접근

"""
히스토그램, 이미지, 텍스트를 시각화할 수도 있고, 오디오를 들을 수도 있다
test_logdir = get_run_logdir()
writer = tf.summary.create_file_writer(str(test_logdir))
with writer.as_default():
    for step in range(1, 1000 + 1):
        tf.summary.scalar("my_scalar", np.sin(step / 10), step=step)

        data = (np.random.randn(100) + 2) * step / 100  # gets larger
        tf.summary.histogram("my_hist", data, buckets=50, step=step)

        images = np.random.rand(2, 32, 32, 3) * step / 1000  # gets brighter
        tf.summary.image("my_images", images, step=step)

        texts = ["The step is " + str(step), "Its square is " + str(step ** 2)]
        tf.summary.text("my_text", texts, step=step)

        sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio("my_audio", audio, sample_rate=48000, step=step)
"""

# 케라스 튜너

In [ ]:
#코랩에서 케라스 튜너 설치
%pip install -q -U keras-tuner

In [ ]:
"""
이렇게 쓰는 것도 가능하다. 다만 아래 코드처럼 층 개수는 안정함.
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model
"""

"""
build_model 예시

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=(input_shape,)))

    # 은닉 레이어 수와 각 레이어의 유닛 수를 하이퍼파라미터로 설정
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),
                                     min_value=32, max_value=512, step=32),
                                     activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer= hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop']) # 여러가지 옵티마이저 고르기
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

    또는 학습률을 조정하기
    model.compile(optimizer=keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

"""


"""
학습률을 동적으로 조정하는 하이퍼파라미터 스케줄링도 있다. arxiv 1711.09846
학습률, 여러 가지 드랍아웃 rate를 조정할  수 있다.
GAN에서는 생성자와 판별자의 학습률을 조정한다던지..

ExponentialDecay: 매 스텝마다 학습률이 decay_rate만큼 곱해지며,
                  일반적으로 decay_steps 마다 업데이트됩니다.
데이터 샘플의 개수가 60000개이고, 미니배치 사이즈가 32이고 decay_steps가 10000일때
한 에포크당 60000 / 32 = 1875개의 배치가 처리되므로
10000 / 1875 = 5.53에포크마다 학습률이 조정된다.


initial_learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='binary_crossentropy',
              metrics=['accuracy'])

"""

In [ ]:
import tensorflow as tf
import keras_tuner as kt

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

def build_model(hp): #hp = 하이퍼 파라미터
# 이렇게 n_hidden 파라미터를 만드는데, 이 파라미터가 내부에 없으면
# 자동으로 추가한다.

    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2) # 층의 개수
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)  # 유닛의 개수
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, # 학습률 조정
                             sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"]) #옵티마이저 고르기(learning_rate를 적용하기 위해 아래처럼 구성함)
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model

random_search_tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=5, overwrite=True,
    directory="my_fashion_mnist", project_name="my_rnd_search", seed=42)
random_search_tuner.search(X_train, y_train, epochs=10,
                           validation_data=(X_valid, y_valid))

Trial 5 Complete [00h 02m 23s]
val_accuracy: 0.8416000008583069

Best val_accuracy So Far: 0.8641999959945679
Total elapsed time: 00h 09m 07s


In [ ]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values  # best hyperparameter values

{'n_hidden': 8,
 'n_neurons': 37,
 'learning_rate': 0.008547485565344062,
 'optimizer': 'sgd'}

In [ ]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 3 summary
Hyperparameters:
n_hidden: 8
n_neurons: 37
learning_rate: 0.008547485565344062
optimizer: sgd
Score: 0.8641999959945679


In [ ]:
best_trial.metrics.get_last_value("val_accuracy") #가장 좋은 점수 확인

0.8641999959945679

In [ ]:
#마지막 훈련
#best_model = random_search_tuner.get_best_models()
best_model.fit(X_train_full, y_train_full, epochs=10)
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)

In [ ]:
"""
상황에 따라 데이터 전처리 하이퍼파라미터 또는
배치 크기와 같은 model.fit() 매개변수를 미세 조정해야 할 수 있는데
이때는 사용자 정의 모델을 만들어서 튜너에 전달해야 한다.
"""
# 정규화를 추가한 모델
class MyClassificationHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp) #위에서 만든 모델

    def fit(self, hp, model, X, y, **kwargs): #fit 메서드만 새로 만듬
        if hp.Boolean("normalize"):
            norm_layer = tf.keras.layers.Normalization()
            X = norm_layer(X)
        return model.fit(X, y, **kwargs)

"""
하이퍼밴드 튜너
몇 번의 에포크동안 여러 모델을 훈련한 다음, 상위 1 / factor(3)의 모델만 남긴다.
이 과정을 단일 모델이 남을때까지 반복한다.
사용 방법은 위의 랜덤 서칭과 거의 동일함.

max_epochs : 각 하이퍼파라미터 설정을 위한 최대 에포크 수
factor : factor가 3이면 각 반복마다 모델 수를 1/3로 줄임.
min_epochs(선택적) : 각 하이퍼파라미터 설정의 최소 에포크 수
objective : 최적화하고자 하는 목표 지표 ('val_accuracy', 'val_loss' 등)
hyperband_iterations(선택적): 전체 hyperband 알고리즘을 몇 번 실행할지
diretory = 파일 경로
project_name = 폴더 이름

"""
hyperband_tuner = kt.Hyperband(
    MyClassificationHyperModel(), objective="val_accuracy", seed=42,
    max_epochs=10, factor=3, hyperband_iterations=2,
    overwrite=True, directory="my_fashion_mnist", project_name="hyperband")

In [ ]:
root_logdir = Path(hyperband_tuner.project_dir) / "tensorboard"
tensorboard_cb = tf.keras.callbacks.TensorBoard(root_logdir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2)
hyperband_tuner.search(X_train, y_train, epochs=10,
                       validation_data=(X_valid, y_valid),
                       callbacks=[early_stopping_cb, tensorboard_cb])

Trial 60 Complete [00h 01m 27s]
val_accuracy: 0.8565999865531921

Best val_accuracy So Far: 0.8784000277519226
Total elapsed time: 00h 41m 12s


In [ ]:
best_trial = hyperband_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 0054 summary
Hyperparameters:
n_hidden: 7
n_neurons: 247
learning_rate: 0.0003987720809096887
optimizer: adam
normalize: True
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0049
Score: 0.8784000277519226


정규화를 추가한 하이퍼밴드 튜너\
학습률 0.00039877\
뉴런 247.00\
정규화 true\
층 7.0000\
옵티마이저 adam\

검증 정확도 : 0.8784000277519226

best_trial = hyperband_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

In [ ]:
"""
하이퍼밴드나, 랜덤검색은 결국 하이퍼파라미터 공간을 랜덤으로 탐색한다는 점 때문에
빠르지만 듬성듬성하다는 단점이 있다. 이것을 보완하기 위한것으로
베이지안 옵티마이저가 있는데,
alpha는 성능 측정에서 예상되는 잡음 수준을
beat는 알고리즘이 얼마나 공간을 탐색할지를 지정한다. (나머지는
"""
bayesian_opt_tuner = kt.BayesianOptimization(
    MyClassificationHyperModel(), objective="val_accuracy", seed=42,
    max_trials=10, alpha=1e-4, beta=2.6,
    overwrite=True, directory="my_fashion_mnist", project_name="bayesian_opt")
bayesian_opt_tuner.search(X_train, y_train, epochs=10,
                          validation_data=(X_valid, y_valid),
                          callbacks=[early_stopping_cb])